# **Projeto de Machine Learning - Banco de Dados do Banco**

Este projeto consiste em aplicar técnicas de Machine Learning em um dataset sobre clientes de um banco, com o objetivo de prever se o cliente vai assinar um depósito a prazo. O código é dividido em etapas para seguir o fluxo recomendado: carregamento, preparação, modelagem, avaliação e otimização.

1. **Baixar e Extrair o Dataset**

Primeiro, vamos fazer o download e a extração do dataset.

In [36]:
import os
import zipfile
import requests
import pandas as pd
import numpy as np
import shutil
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV



# URL do dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
zip_file_path = "bank.zip"
extract_folder = "bank_dataset"

# Baixando e extraindo o dataset
print("Baixando dataset...")
response = requests.get(url)
with open(zip_file_path, "wb") as f:
    f.write(response.content)

print("Extraindo arquivos...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Definindo caminho do arquivo CSV
csv_file = os.path.join(extract_folder, "bank-full.csv")

# Carregando o dataset
print("Carregando o dataset...")
df = pd.read_csv(csv_file, sep=";")

# Exibindo as primeiras linhas do dataset
print("Dataset carregado com sucesso!")
print(df.head())

Baixando dataset...
Extraindo arquivos...
Carregando o dataset...
Dataset carregado com sucesso!
   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  


2. **Preparação e Pré-processamento dos Dados**

Aqui, vamos tratar valores ausentes, codificar variáveis categóricas e separar as variáveis independentes da variável alvo.

In [12]:
# Excluindo colunas desnecessárias
df.drop(columns=['duration'], inplace=True)  # 'duration' não é relevante para o modelo

# Codificando variáveis categóricas
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Separando variáveis independentes e dependentes
X = df.drop('y', axis=1)
y = df['y']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

3. **Construção do Pipeline**

Vamos criar um pipeline para realizar a transformação dos dados e o treinamento do modelo de forma eficiente e modular.

In [13]:
# Criando o pipeline
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Substituindo valores ausentes pela moda
    ('scaler', StandardScaler())  # Normalizando as variáveis para melhorar o desempenho do modelo
])

4. **Treinamento e Avaliação dos Modelos**

Aqui, treinamos três modelos diferentes: Random Forest, Logistic Regression, e SVM, e comparamos seus desempenhos.

In [14]:
# Modelos a serem testados
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(random_state=42)
}

# Treinamento, avaliação e comparação dos modelos
for model_name, model in models.items():
    print(f"\nTreinando e avaliando o modelo: {model_name}")

    # Pipeline completo com o modelo
    model_pipeline = Pipeline(steps=[
        ('preprocessor', pipeline),
        ('classifier', model)
    ])

    # Cross-validation
    cv_scores = cross_val_score(model_pipeline, X_train, y_train, cv=5, scoring='accuracy')
    print(f"Cross-validation scores: {cv_scores}")
    print(f"Average accuracy: {np.mean(cv_scores):.4f}")

    # Treinando no conjunto de treinamento
    model_pipeline.fit(X_train, y_train)

    # Avaliação no conjunto de teste
    y_pred = model_pipeline.predict(X_test)
    print("\nMétricas de avaliação:")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


Treinando e avaliando o modelo: Random Forest
Cross-validation scores: [0.89225908 0.89368088 0.89145205 0.88987202 0.89066203]
Average accuracy: 0.8916

Métricas de avaliação:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     11966
           1       0.63      0.21      0.31      1598

    accuracy                           0.89     13564
   macro avg       0.77      0.60      0.63     13564
weighted avg       0.87      0.89      0.87     13564

[[11769   197]
 [ 1265   333]]

Treinando e avaliando o modelo: Logistic Regression
Cross-validation scores: [0.88309637 0.88293839 0.8832359  0.88212988 0.88276189]
Average accuracy: 0.8828

Métricas de avaliação:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     11966
           1       0.45      0.01      0.02      1598

    accuracy                           0.88     13564
   macro avg       0.67      0.50      0.48     13564
weighted

5. **Otimização de Hiperparâmetros**

Agora, vamos realizar a otimização de hiperparâmetros para o modelo Random Forest usando o GridSearchCV. O objetivo é encontrar os melhores parâmetros para o modelo, melhorando a sua performance e evitando overfitting.



In [23]:
# Definição do pipeline de pré-processamento
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())  # Padronização das variáveis
])

# Parâmetros para GridSearchCV
param_grid_rf = {
    'classifier__n_estimators': [100, 200],  # Número de árvores
    'classifier__max_depth': [10, 20, None],  # Limitar a profundidade
    'classifier__min_samples_split': [2, 5]  # Exigência de amostras mínimas para divisão
}

# Inicializando o modelo RandomForest
rf_model = RandomForestClassifier(random_state=42)

# Pipeline completo com o modelo RandomForest
model_pipeline = Pipeline(steps=[
    ('preprocessor', pipeline),
    ('classifier', rf_model)
])

# Realizando a busca de hiperparâmetros com GridSearchCV
grid_search = GridSearchCV(model_pipeline, param_grid_rf, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Avaliação no conjunto de teste com o melhor modelo
y_pred = grid_search.predict(X_test)
print("\nMétricas de avaliação com o melhor modelo:")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Métricas de avaliação com o melhor modelo:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     11966
           1       0.65      0.20      0.31      1598

    accuracy                           0.89     13564
   macro avg       0.78      0.59      0.63     13564
weighted avg       0.87      0.89      0.87     13564

[[11790   176]
 [ 1274   324]]


6. **Conclusão**

Esse código ajuda a prever quais clientes, no conjunto de dados, irão assinar o depósito a prazo, adicionando essa previsão como uma nova coluna no dataset original. O dataset atualizado pode ser exportado para um arquivo CSV e usado para tomar decisões de negócios, como direcionar ofertas de depósitos a prazo para os clientes identificados como prováveis assinantes.

In [37]:
# Prevendo se os clientes irão assinar ou não o depósito a prazo
y_pred_final = grid_search.predict(X)  # Prevendo para todos os clientes, não apenas X_test

# Adicionando a previsão ao dataset original
df['assinaram_deposito'] = y_pred_final  # 1 para quem irá assinar, 0 para quem não vai

# Substituindo 1 por 'sim' e 0 por 'não' na coluna 'assinaram_deposito'
df['assinaram_deposito'] = df['assinaram_deposito'].replace({1: 'sim', 0: 'não'})

# Exibindo o dataset completo com a nova coluna 'assinaram_deposito'
print(df)

# Salvar o arquivo CSV localmente no Colab
df.to_csv("bank_com_assinatura.csv", index=False)

# Baixar o arquivo para o seu computador
files.download("bank_com_assinatura.csv")

       age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
3       47   blue-collar   married    unknown      no     1506     yes   no   
4       33       unknown    single    unknown      no        1      no   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  secondary      no      668      no   no   
45210   37  entrepreneur   married  secondary      no     2971      no   no   

         contact  day month  duration  campaign  pd

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>